For some strange reason, the VEST only has the precinct shapefiles for VA in 2019, not the election results. I'm going to have to merge them in myself. 

In [13]:
library(sf)
library(ggplot2)
library(dplyr)
library(tibble)
library(magrittr)
library(tidyverse)

tabulate_votes <- function(votes_f_name, precincts_f_name, out_f_name) {
    # read csv into dataframe
    data_dir = "C:/Users/madie/OneDrive/data/"
    df <- read.csv(file = paste(data_dir, votes_f_name, sep=""))

    df <- df %>% 
        # remove all rows that aren't from an HOD election
        filter(DistrictType == "House of Delegates") %>%
        # remove all third parties
        filter(Party %in% c("Democratic", "Republican")) %>%
        # remove all provisional ballots
        filter(!(PrecinctName %in% c("# AB - Central Absentee Precinct", "## Provisional")))

    # generate summary table with precincts as rows and repub and dem votes as cols
    df_votes <- df %>%
        group_by(PrecinctName, LocalityName) %>%
        summarise(demvote = sum(TOTAL_VOTES[Party == "Democratic"]),
            repvote = sum(TOTAL_VOTES[Party == "Republican"])) %>%
        distinct() 

    #split precinctname into precinct and precinct ID
    df_votes <- df_votes[order(df_votes$PrecinctName),] %>%
        separate(col = PrecinctName, sep = " - ", into = c("precinctID", "precinct")) %>%
        # remove precinct ID
        subset(select = -precinctID)

    # read in precincts shapefile
    df_shp <- st_read(paste(data_dir, precincts_f_name, sep="")) %>%
        # rename PRECINCT col
        rename(precinct = PRECINCT) %>% 
        # remove the random z axis (idk why it's there)
        st_zm(drop = TRUE, what = "ZM")

    # make precinct names in shp file all uppercase
    df_shp$precinct <- df_shp$precinct %>% toupper()

    # join election results into precinct
    df_shp <- df_shp %>%
        left_join(df_votes, by = "precinct")

    # write to file
    st_write(obj=df_shp, dsn=paste(data_dir, out_f_name, sep=""), append=FALSE)
}

In [14]:
votes_f_name = "official-VA-2005-2019/G19.csv"
precincts_f_name = "vest/va_2019_statehouse/va_2019_statehouse.shp"
out_f_name = "vest/va_2019_statehouse_full/va_2019_statehouse_full.shp"


tabulate_votes(votes_f_name, precincts_f_name, out_f_name)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

Warning message:
"Expected 2 pieces. Additional pieces discarded in 9 rows [439, 810, 943, 1175, 1548, 1621, 1634, 2318, 2389]."
Warning message:
"Expected 2 pieces. Missing pieces filled with `NA` in 13 rows [25, 66, 103, 238, 242, 583, 731, 843, 981, 1475, 1486, 2181, 2245]."


Reading layer `va_2019_statehouse' from data source `C:\Users\madie\OneDrive\data\vest\va_2019_statehouse\va_2019_statehouse.shp' using driver `ESRI Shapefile'
Simple feature collection with 2586 features and 5 fields
geometry type:  MULTIPOLYGON
dimension:      XYZ
bbox:           xmin: -83.67539 ymin: 36.54085 xmax: -75.16643 ymax: 39.46601
z_range:        zmin: 0 zmax: 0
geographic CRS: NAD83


Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"


Deleting layer `va_2019_statehouse_full' using driver `ESRI Shapefile'
Writing layer `va_2019_statehouse_full' to data source `C:/Users/madie/OneDrive/data/vest/va_2019_statehouse_full/va_2019_statehouse_full.shp' using driver `ESRI Shapefile'
Writing 3675 features with 8 fields and geometry type Multi Polygon.
